# 코랩에서 임의의 유튜브 동영상 객체 검출

- YOLO 사용
- GPU 설정해야 함

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/swkim01/DSAC5/blob/main/gg-21-YOLO_YouTube.ipynb"><img src="https://github.com/swkim01/DSAC5/raw/main/colab_logo_32px.png" />구글 코랩에서 실행</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/swkim01/DSAC5/blob/main/gg-21-YOLO_YouTube.ipynb"><img src="https://github.com/swkim01/DSAC5/raw/main/GitHub-Mark-32px.png" />깃헙에서 소스 보기</a>
  </td>
</table>


 


In [ ]:
!pip install pydrive
!git clone https://github.com/thtrieu/darkflow.git
!pip install Cython 
!pip install pytube
!pip install PyDrive
!pip install tensorflow==1.2
!pip install opencv-python
import os
os.chdir('/content/darkflow')
!pwd

!pip install -e .

Cloning into 'darkflow'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 2713 (delta 0), reused 0 (delta 0), pack-reused 2709
Receiving objects: 100% (2713/2713), 32.98 MiB | 43.02 MiB/s, done.
Resolving deltas: 100% (1761/1761), done.
     |████████████████████████████████| 40kB 5.1MB/s 


In [ ]:
!pwd

/content/darkflow


### 아래 링크에서 가중치를 다운로드 하여 구글 드라이브에 저장한다.

- YOLOv2 608x608을 사용한다. 
- https://pjreddie.com/darknet/yolo/
- 마우스 우측 클릭으로 파일 id를 확인한다.

In [ ]:
import pydrive
import time
from tqdm import tqdm
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# load training weights

newpath = '/content/darkflow/bin'
if not os.path.exists(newpath):
  os.makedirs(newpath)

download = drive.CreateFile({'id': '1_8qWuzgT91U9f11jsCoGWhkf3Bled5rZ'}) # insert the fild ID here
download.GetContentFile('/content/darkflow/bin/yolov2.weights')



In [ ]:
from darkflow.net.build import TFNet
import cv2
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:461: FutureWarning: Passing (type, 1) or 

In [ ]:
options = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolov2.weights',
    'threshold': 0.3,
    'gpu' : 1.0
          }

In [ ]:
tfnet = TFNet(options)

/content/darkflow/darkflow/dark/darknet.py:54: UserWarning: ./cfg/yolov2.cfg not found, use cfg/yolo.cfg instead
  cfg_path, FLAGS.model))


Parsing cfg/yolo.cfg
Loading bin/yolov2.weights ...
Successfully identified 203934260 bytes
Finished in 0.08666253089904785s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!  | conv 3x3p1_1  +bn

In [ ]:
def predict_img(link):
  
  result = tfnet.return_predict(link)
   

  for n,value in enumerate(result):
    
    tl = (result[n]['topleft']['x'], result[n]['topleft']['y'])
    br = (result[n]['bottomright']['x'], result[n]['bottomright']['y'])
    label = result[n]['label']
    img = cv2.rectangle(img,tl,br,(0,255,0))
    
  plt.figure(figsize=(20,10))    
  plt.imshow(img)

In [ ]:
def create_image_paths():
  
  import glob
  # Get all image file paths to a list.
  images = list(glob.iglob(os.path.join('/content/sample_data_frames/',title, '*.*')))
  # Sort the images by name index.
  images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))
  
  return images

In [ ]:
def download_youtube_video(link):
  
  from pytube import YouTube
  yt = YouTube(link)
  streams = yt.streams.get_by_itag('18')
  streams.download('/content/sample_data')
  title = streams.default_filename
  
  print('Youtube Video Downloaded...')
  return  title

In [ ]:
def fps():
  video = cv2.VideoCapture(os.path.join('/content/sample_data/',title))  
  fps = video.get(cv2.CAP_PROP_FPS)
  print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
  video.release()
  return fps

In [ ]:
def make_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
    """
    from tqdm import tqdm
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in tqdm(images):
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

#outvid = os.path.join(VIDEO_DIR, "out.mp4")
#make_video('/content/sample_data/video_out.mp4', images, fps=30)


In [ ]:
def sample_frames():
  
  capture = cv2.VideoCapture(os.path.join('/content/sample_data/',title)) 

  
  frames = []
  frame_count = 0

  video_frame_path = os.path.join('/content/sample_data_frames/',title)
  
  if not os.path.exists(video_frame_path):
    os.makedirs(video_frame_path)
    print('Frames Directory Created...')
  
  while True:
  
    ret, frame = capture.read()
    frames.append(frame)
    frame_count += 1
  
    if not ret:
      print(frame_count)
      return (frames,frame_count,video_frame_path)
      break

In [ ]:
def predict_video(link):
  
  global title
  
  title = download_youtube_video(link)
  frames,frame_count,video_frame_path = sample_frames()
  
  #load video and get every frame of it for analysis..
  # for every frame run yolo prediction and save the frame as jpg file

  for count,frame in tqdm(enumerate(frames[1:-10])):
    
    #frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    results = tfnet.return_predict(frame)
  
    for n,value in enumerate(results):
      
      tl = (results[n]['topleft']['x'], results[n]['topleft']['y'])
      br = (results[n]['bottomright']['x'], results[n]['bottomright']['y'])
      label = results[n]['label']
      frame = cv2.rectangle(frame,tl,br,(0,255,0),3)
      frame = cv2.putText(frame,label,tl,cv2.FONT_HERSHEY_COMPLEX,1,(0,0,0),2)
      
      #frame = cv2.addText(label)
      #frame = cv2.line(img,(0,frame.shape[0]//2),(600,img.shape[0]//2),(255,0,0),2)   
      
  
    name = '{0}.jpg'.format(count)
    
    name = os.path.join(video_frame_path,name)  
    cv2.imwrite(name, frame)
    
  print('All Frames Saved...')
  
  newpath = '/content/sample_data_out/'
  if not os.path.exists(newpath):
    os.makedirs(newpath)
    
  images = create_image_paths()
  #print(images)
  frames_per_second = fps()
  make_video(os.path.join('/content/sample_data_out/',title), images, fps=frames_per_second)
  
  
    
  
  
  

## 유튜브 비디오 링크를 입력하고 YOLO를 실행
- 해상도 360p를 권장한다 (colab 사용 한계)


In [ ]:

# predict_video('https://www.youtube.com/watch?v=wlJzRAUexV0')
# predict_video('https://www.youtube.com/watch?v=fM5aHIVBTIc')
predict_video('https://www.youtube.com/watch?v=SDTZ7iX4vTQ')

Youtube Video Downloaded...



0it [00:00, ?it/s]

6135



1it [00:04,  4.46s/it]
2it [00:08,  4.44s/it]
3it [00:13,  4.42s/it]
4it [00:17,  4.39s/it]
5it [00:21,  4.36s/it]
6it [00:26,  4.37s/it]
7it [00:30,  4.37s/it]
8it [00:35,  4.38s/it]
9it [00:39,  4.42s/it]
10it [00:43,  4.44s/it]
11it [00:48,  4.42s/it]
12it [00:52,  4.41s/it]
13it [00:57,  4.41s/it]
14it [01:01,  4.42s/it]
15it [01:06,  4.42s/it]
16it [01:10,  4.43s/it]
17it [01:14,  4.43s/it]
18it [01:19,  4.41s/it]
19it [01:23,  4.44s/it]
20it [01:28,  4.45s/it]
21it [01:32,  4.46s/it]
22it [01:37,  4.46s/it]
23it [01:41,  4.47s/it]
24it [01:46,  4.46s/it]
25it [01:50,  4.47s/it]
26it [01:55,  4.48s/it]
27it [01:59,  4.49s/it]
28it [02:04,  4.50s/it]
29it [02:08,  4.49s/it]
30it [02:13,  4.51s/it]
31it [02:17,  4.48s/it]
32it [02:22,  4.46s/it]
33it [02:26,  4.46s/it]
34it [02:30,  4.44s/it]
35it [02:35,  4.44s/it]
36it [02:39,  4.44s/it]
37it [02:44,  4.46s/it]
38it [02:48,  4.47s/it]
39it [02:53,  4.50s/it]
40it [02:57,  4.50s/it]
41it [03:02,  4.50s/it]
42it [03:06,  4.49s/it]


KeyboardInterrupt: ignored

In [ ]:
files.download(os.path.join('/content/sample_data_out/',title))
